# Naive Bayes Classifier

## Perform LED digit classification using Naive bayes classifier  in python.

We will represent each digit as a 7-dimensional binary vector where a 1 in the representation implies the corresponding segment to be on. The representations for all ten digits, 0-9, is shown below.Furthermore, we assume the display to be faulty in the sense that with probability p a segment doesn’t turn on(off) when it is supposed to be on(off). Thus, we want to design a naive Bayes classifier that accepts a 7-dimensional binary vector as an input and predicts the digit that was meant to be displayed.

|s1|s2|s3|s4|s5|s6|s7|Digit-ID|
|--|--|--|--|--|--|--|--------|
|0 |1 |1 |0 |0 |0 |0 |1       |
|1 |1 |0 |1 |1 |0 |1 |2       |
|1 |1 |1 |1 |0 |0 |1 |3       |
|0 |1 |1 |0 |0 |1 |1 |4       |
|1 |0 |1 |1 |0 |1 |1 |5       |
|0 |0 |1 |1 |1 |1 |1 |6       |
|1 |1 |1 |0 |0 |0 |0 |7       |
|1 |1 |1 |1 |1 |1 |1 |8       |
|1 |1 |1 |0 |0 |1 |1 |9       |
|1 |1 |1 |1 |1 |1 |0 |0       |

In [1]:
import pandas as pd
import numpy as np
import random
random.seed(3)

### Lets consider some noise level 
Noiselevel in the formula refers to a cell where we store the probabilty p of a segment being faulty.

In [2]:
noise_level = 0.1

Lets store the segment values of each digit in an array for later computation purpose.

In [3]:
dig1 = [0,1,1,0,0,0,0]
dig2 = [1,1,0,1,1,0,1]
dig3 = [1,1,1,1,0,0,1]
dig4 = [0,1,1,0,0,1,1]
dig5 = [1,0,1,1,0,1,1]
dig6 = [0,0,1,1,1,1,1]
dig7 = [1,1,1,0,0,0,0]
dig8 = [1,1,1,1,1,1,1]
dig9 = [1,1,1,0,0,1,1]
dig0 = [1,1,1,1,1,1,0]

digs = [dig0, dig1, dig2, dig3, dig4, dig5, dig6,dig7, dig8, dig9]

### Generate training dataset

Lets use python rand function to generate a random float between 0 and 1, and if that number is greater than the given noise level we keep the segment as same, but if not we flip the segment, i.e., 0 to 1 or 1 to 0. The below function generates segment representation of digits based on the noise level and the input digit original segment representation.

In [4]:
def generateDigit(digit, noise_level):
    arr = []
    for segment in digit:
        rand = random.random()
        if(rand > noise_level):
            arr.append(segment)
        else:
            arr.append(1 - segment)
    return arr

Now lets generate the training data set with each digit containting 100 examples each.

In [5]:
examples = 100

In [6]:
training_set = []
for dig in digs:
    arr = []
    for example in range(examples):
        arr.append(generateDigit(dig, noise_level))
    training_set.append(arr)

For visualisation lets print the training data set with lets say starting 5 examples each

In [7]:
pd.DataFrame(training_set).iloc[:, :5]

,0,1,2,3,4
0,"[1, 1, 1, 1, 1, 0, 1]","[1, 1, 1, 1, 1, 1, 0]","[1, 1, 1, 1, 1, 1, 0]","[0, 1, 1, 1, 0, 1, 0]","[1, 1, 1, 1, 1, 1, 0]"
1,"[1, 1, 1, 0, 0, 0, 1]","[0, 1, 0, 0, 0, 0, 1]","[0, 1, 1, 0, 0, 0, 0]","[0, 1, 1, 0, 1, 0, 0]","[1, 1, 1, 0, 1, 0, 0]"
2,"[1, 1, 0, 1, 1, 0, 1]","[1, 0, 0, 0, 1, 0, 1]","[0, 1, 0, 1, 1, 0, 1]","[1, 0, 0, 0, 1, 0, 0]","[1, 1, 0, 1, 1, 1, 1]"
3,"[1, 1, 1, 1, 0, 0, 1]","[1, 1, 1, 1, 0, 0, 1]","[1, 0, 1, 1, 1, 0, 1]","[0, 1, 0, 1, 0, 0, 1]","[1, 1, 1, 1, 1, 1, 1]"
4,"[0, 1, 1, 0, 0, 1, 1]","[0, 1, 1, 0, 0, 1, 1]","[0, 1, 1, 1, 0, 1, 1]","[0, 0, 1, 0, 0, 1, 1]","[0, 1, 1, 0, 0, 1, 1]"
5,"[1, 1, 0, 1, 0, 1, 0]","[1, 0, 1, 1, 0, 1, 1]","[1, 0, 1, 1, 0, 1, 1]","[1, 0, 0, 1, 0, 1, 1]","[1, 0, 1, 1, 0, 1, 1]"
6,"[0, 0, 1, 1, 1, 1, 1]","[0, 1, 1, 1, 1, 1, 1]","[0, 0, 1, 1, 1, 1, 1]","[0, 0, 1, 0, 1, 1, 1]","[0, 0, 1, 1, 1, 1, 1]"
7,"[1, 1, 1, 0, 0, 0, 0]","[1, 1, 1, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0, 0]","[1, 1, 1, 0, 0, 0, 0]","[1, 1, 1, 0, 0, 1, 0]"
8,"[1, 1, 1, 1, 1, 1, 1]","[0, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 0, 1, 1]","[1, 1, 1, 1, 1, 1, 0]","[1, 1, 1, 1, 1, 0, 1]"
9,"[1, 0, 1, 0, 0, 1, 1]","[1, 0, 1, 0, 0, 1, 1]","[1, 1, 1, 0, 0, 1, 1]","[1, 1, 1, 0, 0, 1, 1]","[1, 1, 1, 0, 0, 1, 0]"


### Naive Bayes Classification

A Naive Bayes (NB) classifier uses Bayes’ theorem and independent features assumption to perform classification. Although the feature independence assumption may not hold true, the resulting simplicity and performance close to complex classifiers offer complelling reasons to treat features to be independent. Suppose we have $d$ features, $x_1,\cdots, x_d$, and two classes $c_1\text{ and } c_2$. According to Bayes’ theorem, the probability that the observation vector ${X} = [x_1,\cdots,x_d]^T$ belongs to class $c_j$ is given by the following relationship:

$$P(C_j|x) = \frac{P(x_1,...,x_d|c_j)P(c_j)}{P(x_1,...,x_d)}, j=1,2$$ 
Assuming features to be independent, the above expression reduces to: 
$$P(C_j|x) = \frac{P(c_j)\prod_{i=1}^d P(x_i|c_j)}{P(x_1,...,x_d)}, j=1,2$$

The denominator in above expression is constant for a given input. Thus, the classification rule for a given observation vector can be expressed as:

Assign

$x\rightarrow c_1\text{ if } P(c_1)\prod_{i=1}^d P(x_i|c_1) \geq P(c_2)\prod_{i=1}^d P(x_i|c_2) $

Otherwise

$x\rightarrow c_2$

For classification problems with C classes, we can write the classification rule as:

$x\rightarrow c_j \text{ where } P(c_j)\prod_{i=1}^d P(x_i|c_j) > P(c_k)\prod_{i=1}^d P(x_i|c_k), k=1,..,C \text{ and }  k\neq j  $

### Naive bayes design for faulty digit recognition

Designing NB classifier means we need to compute/estimate class priors and conditional probabilities. Class priors are taken as the fraction of examples from each class in the training set. In the present case, all class priors are equal. This means that class priors do not play any role in arriving at the class membership decision in our present example. Thus, we need to estimate only conditional probabilities. The conditional probabilities are the frequencies of each attribute value for each class in our training set. The following relationship provides us with the probability of segment 1 being equal to 1 conditioned on that the digit being displayed is digit 1.

$$P(s_1 = 1|digit1) = \frac{\text{ count of 1's for segment 1 in digit 1 training examples }}{\text{ number of digit 1 training examples}}$$

Since only two possible states, 1 and 0, are possible for each segment, we can calculate the probability of segment 1 being equal to 0 conditioned on that the digit being displayed is digit 1 by the following relationship:

$$P(s_1 = 0|digit1) = 1 - P(s_1 = 1|digit1)$$

In practice, however, a correction is applied to conditional probabilities calculations to ensure that none of the probabilities is 0. This correction, known as Laplace smoothing, is given by the following relationship:

$$P(s_1 = 1|digit1) = \frac{\text{1 + count of 1's for segment 1 in digit 1 training examples }}{\text{2 + number of digit 1 training examples}}$$

Adding 1 to the numerator count ensures probability value doesnot become 0. Adding 2 to the denominator reflects the number of states that are possible for the attribute under consideration. In this case we have binary attributes.

> **So for our case lets calculate the probabilities of each segment 1 with their respective digits producing $10\;digits\times7\;segments$ matrix**

In [8]:
prob = []
for dig in range(10):
    arr = [0]*7
    t_set = training_set[:][dig]
    for row in t_set:
        for index, segment in enumerate(row):
            if(segment == 1):
                arr[index] += 1
    for index, value in enumerate(arr):
        arr[index] = (value+1)/(2+examples)
    prob.append(arr)
df = pd.DataFrame(prob)
df.rename(columns={0: "s1", 1: "s2",2:  "s3", 3:"s4", 4: "s5", 5: "s6", 6: "s7"}, inplace=True)
df

,s1,s2,s3,s4,s5,s6,s7
0,0.892157,0.921569,0.862745,0.931373,0.862745,0.843137,0.088235
1,0.107843,0.960784,0.901961,0.117647,0.127451,0.127451,0.117647
2,0.882353,0.921569,0.078431,0.882353,0.882353,0.117647,0.911765
3,0.882353,0.892157,0.921569,0.872549,0.088235,0.039216,0.892157
4,0.127451,0.882353,0.882353,0.098039,0.107843,0.911765,0.911765
5,0.882353,0.147059,0.862745,0.911765,0.088235,0.882353,0.852941
6,0.088235,0.107843,0.843137,0.872549,0.911765,0.901961,0.921569
7,0.911765,0.892157,0.882353,0.098039,0.068627,0.127451,0.098039
8,0.823529,0.901961,0.852941,0.901961,0.931373,0.901961,0.901961
9,0.862745,0.882353,0.950980,0.078431,0.078431,0.852941,0.794118


### Testing the model

Having calculated conditional probabilities, we are now ready to see how well our classifier will work on test examples. For this, we first generate three test examples for each digit following the steps outlined earlier

In [9]:
examples_test = 3

In [10]:
test_set = []
test_labels = []
for index, dig in enumerate(digs):
    arr = []
    labels = [index]*examples_test
    for example in range(examples_test):
        arr.append(generateDigit(dig, noise_level))
    test_set.append(arr)
    test_labels.append(labels)     
pd.DataFrame(test_set)

,0,1,2
0,"[1, 1, 1, 0, 1, 1, 0]","[1, 1, 1, 1, 1, 1, 0]","[0, 1, 1, 1, 1, 1, 0]"
1,"[0, 1, 1, 0, 0, 0, 0]","[0, 1, 1, 0, 0, 0, 1]","[0, 1, 1, 0, 0, 0, 0]"
2,"[1, 1, 0, 1, 1, 0, 1]","[1, 1, 0, 1, 0, 0, 1]","[0, 1, 0, 1, 1, 0, 1]"
3,"[1, 1, 1, 1, 0, 0, 1]","[1, 1, 1, 1, 0, 0, 1]","[0, 0, 0, 1, 0, 1, 1]"
4,"[0, 1, 1, 0, 0, 1, 1]","[0, 1, 1, 0, 0, 1, 1]","[0, 1, 1, 0, 0, 1, 1]"
5,"[0, 0, 1, 1, 0, 1, 1]","[1, 0, 1, 1, 0, 1, 1]","[1, 0, 1, 1, 1, 0, 1]"
6,"[0, 0, 1, 1, 1, 1, 1]","[0, 0, 1, 1, 1, 1, 1]","[0, 0, 0, 1, 1, 1, 1]"
7,"[1, 1, 1, 0, 0, 0, 0]","[1, 1, 1, 0, 0, 0, 0]","[1, 1, 1, 0, 0, 0, 0]"
8,"[0, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1]","[1, 1, 0, 1, 1, 1, 1]"
9,"[1, 1, 1, 0, 0, 1, 1]","[1, 1, 1, 0, 0, 1, 1]","[1, 1, 1, 0, 0, 0, 1]"


Since we got the test data set, lets calculate the probabilities of each example using naive bayes. Here using above computed probability matrix we perform conditional probability for each digit on set of 7 segments in each example and pick the max probable class as class of our example segment set. (Here class means digit)

In [11]:
pred_labels = []
for index_dig, dig in enumerate(test_set):
    l = []
    for example in dig:
        prob_arr = []
        for num in range(10):
            p = 1
            for index_seg, seg in  enumerate(example):
                if (seg == 1):
                    p *= prob[num][index_seg]
                else:
                    p *= (1-prob[num][index_seg])
            prob_arr.append(p)
        l.append(prob_arr.index(max(prob_arr)))
    pred_labels.append(l)
pred_labels

[[0, 0, 0],
 [1, 1, 1],
 [2, 2, 2],
 [3, 3, 5],
 [4, 4, 4],
 [5, 5, 3],
 [6, 6, 6],
 [7, 7, 7],
 [8, 8, 8],
 [9, 9, 3]]

## Accuracy

As we got out predicted and test labels, lets just find the simple accuracy score by comparing both of them and dividing it by the total number of example test cases. This gives us the accuracy of our model

In [12]:
A = np.array(test_labels)
B = np.array(pred_labels)
accuracy = (A == B).sum()/(examples_test*10)
print(f"Accuracy score of our model is: {accuracy}")

Accuracy score of our model is: 0.9
